# Imports

In [92]:
import spacy
import pandas as pd
import matplotlib.pyplot as mltp
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm

Pyteseract --> ausgabe der inhalte der lokalitäten --> basierend davon die inhalte rausmacht --> coordinaten der paragrapthen oder der seitenzahlen zu erkennen 
whaitet baises sehen



# processing

In [93]:
filename='data/AFD_Wahlprogramm_2021.txt'

In [94]:
with open(filename) as f:
    text = f.read()
    #text = text.replace(r'\\d+\\n', '$')
    #text=   re.sub("\n\n", ".", text)
    #text=   re.sub("\d+.", ".", text)
    text=   re.sub(" \d+\n", ".", text)
    text=   re.sub("\n\d+", " ", text)
    text=   re.sub("\n", " ", text)
sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

In [95]:
text

'Deutschland. Aber normal. normal.  Programm der Alternative für Deutschland für die Wahl zum 20. Deutschen Bundestag  AD.   Inhaltsverzeichnis  Demokratie und Rechtsstaat.Das Volk ist der Souverän.Volksabstimmungen nach Schweizer Modell.Die Gewaltenteilung gewährleisten.Entpolitisierung der Justiz.Verfassungsschutz reformieren.Trennung von Amt und Mandat und Kampf der Ämterpatronage.Macht der Parteien beschränken.Freie Listenwahl und freies Mandat.Verkleinerung der Parlamente.Berufspolitikertum: Mandats- und Amtszeiten begrenzen.Lobbyismus eindämmen. Freiheit und Verantwortung.Bürgerlich-freiheitliche Rechtsordnung.Leistung statt Quote - Freiheit statt Diskriminierung.Schutz der persönlichen Freiheit vor digitalen Übergriffen.Hinweisgeber (Whistleblower) schützen. EU und Europa.Ein Europa der Vaterländer.Zurückführung der Europäischen Union in einen Staatenbund souveräner Staaten. Steuern und Finanzen.Keine EU-Steuern.Für ein Steuersystem mit wenigen Steuerarten und verständlicher Sys

In [96]:
sentences

['Deutschland',
 'Aber normal',
 'normal',
 'Programm der Alternative für Deutschland für die Wahl zum 20',
 'Deutschen Bundestag  AD',
 'Inhaltsverzeichnis  Demokratie und Rechtsstaat',
 'Das Volk ist der Souverän',
 'Volksabstimmungen nach Schweizer Modell',
 'Die Gewaltenteilung gewährleisten',
 'Entpolitisierung der Justiz',
 'Verfassungsschutz reformieren',
 'Trennung von Amt und Mandat und Kampf der Ämterpatronage',
 'Macht der Parteien beschränken',
 'Freie Listenwahl und freies Mandat',
 'Verkleinerung der Parlamente',
 'Berufspolitikertum: Mandats- und Amtszeiten begrenzen',
 'Lobbyismus eindämmen',
 'Freiheit und Verantwortung',
 'Bürgerlich-freiheitliche Rechtsordnung',
 'Leistung statt Quote - Freiheit statt Diskriminierung',
 'Schutz der persönlichen Freiheit vor digitalen Übergriffen',
 'Hinweisgeber (Whistleblower) schützen',
 'EU und Europa',
 'Ein Europa der Vaterländer',
 'Zurückführung der Europäischen Union in einen Staatenbund souveräner Staaten',
 'Steuern und Fin

In [97]:
text=sentences

In [98]:
len(text)

2086

In [99]:
text[0:20:-1]

[]

In [100]:
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.de.examples import sentences 
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pickle 

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [101]:
german_stop_words= stopwords.words('german')

In [102]:
def lemmatization_test(texts,  allowed_posttags=['NOUN','ADJ','VERB','ADV']):
    nlp=spacy.load('de_core_news_lg',disable=['parser','ner'])
    texts_out=[]
    for text in tqdm(texts):
        doc= nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=' '.join(new_text)
        texts_out.append(final)
    return texts_out

In [103]:
lemma_text=lemmatization_test(text)
lemma_text_2=lemmatization_test(text,allowed_posttags=['NOUN'])

100%|██████████| 2086/2086 [00:04<00:00, 463.07it/s]


In [104]:
lemma_text = list(filter(None, lemma_text))
lemma_text_2 = list(filter(None, lemma_text_2))

In [105]:
def gen_words(texts):
    final=[]
    for text in tqdm(texts):
        new= gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return final

In [106]:
data_words=gen_words(lemma_text)
data_words_2=gen_words(lemma_text_2)

100%|██████████| 1860/1860 [00:00<00:00, 70207.66it/s]


In [107]:
bigram_phrases=gensim.models.Phrases(data_words,min_count=3,threshold=100)
trigram_phases=gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram=gensim.models.phrases.Phraser(bigram_phrases)
trigram=gensim.models.phrases.Phraser(trigram_phases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trgram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams=make_bigrams(data_words)
data_bigrams_trigrams=make_trgram(data_bigrams)

In [108]:
bigram_phrases_2=gensim.models.Phrases(data_words_2,min_count=3,threshold=100)
trigram_phases_2=gensim.models.Phrases(bigram_phrases[data_words_2],threshold=50)

bigram_2=gensim.models.phrases.Phraser(bigram_phrases_2)
trigram_2=gensim.models.phrases.Phraser(trigram_phases_2)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trgram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams_2=make_bigrams(data_words_2)
data_bigrams_trigrams_2=make_trgram(data_bigrams_2)

In [110]:
from gensim.models import TfidfModel

id2word=corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus=[id2word.doc2bow(text) for text in texts]

print(corpus[3])

tfidf=TfidfModel(corpus,id2word=id2word)

low_value=0.03
words=[]
words_missing_in_tfdf=[]

for i in tqdm(range(0,len(corpus))):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops=low_value_words+words_missing_in_tfdf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus[i]=new_bow


[(4, 1), (5, 1)]


100%|██████████| 1901/1901 [00:00<00:00, 39946.15it/s]


In [111]:
topics=['climate',
'economy',
'education',
'health',
'infrastructure',
'science',
'social causes',
'politics and ideology',
'technology and entrepreneurship']

In [112]:
num_topics=len(topics)

In [113]:
lda_model=gensim.models.ldamodel.LdaModel(
corpus=corpus,
id2word=id2word,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto'
)

In [114]:
dic={}
dic_sum={}
for idx, topic in lda_model.print_topics(num_topics,num_words=100):
    elements=[]
    percent=[]

    for e in topic.split('+'):
        elements.append(e.split('*')[1].replace('"','').strip())
        percent.append(float(e.split('*')[0].replace('"','').strip()))

    dic[str(idx)+'_word']=elements
    dic[str(idx)+'_per']=percent
    dic_sum[idx]=sum(percent)
print(dic_sum)

{0: 0.5030000000000003, 1: 0.47600000000000026, 2: 0.49400000000000027, 3: 0.4700000000000003, 4: 0.5220000000000004, 5: 0.4940000000000003, 6: 0.6790000000000003, 7: 0.49500000000000033, 8: 0.5420000000000004}


In [115]:
pd.DataFrame(data=dic)

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,5_word,5_per,6_word,6_per,7_word,7_per,8_word,8_per
0,deutsch,0.057,offentlich,0.026,wieder,0.020,landwirtschaft,0.025,fordern,0.074,setzen,0.032,paypal,0.115,ausbau,0.020,auch,0.063
1,gut,0.027,erhalten,0.023,dazu,0.015,regional,0.022,nur,0.030,ein,0.025,alternativefuer,0.115,digital,0.020,lehnen,0.033
2,kultur,0.018,frei,0.016,insbesondere,0.013,mensch,0.018,familie,0.020,deshalb,0.015,spenden,0.115,eu,0.017,hoch,0.019
3,sprache,0.014,neu,0.013,ermoglichen,0.013,europaisch,0.015,bleiben,0.017,wald,0.012,burger,0.020,versorgung,0.017,fuhren,0.017
4,schule,0.013,nutzung,0.012,landlich,0.013,starken,0.015,daher,0.016,dafur,0.011,land,0.013,klima_energie_technik_digitalisierung,0.016,sozial,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,stellung,0.002,bitte,0.002,bzw,0.002,konventionell,0.002,last,0.002,gesondert,0.002,ungerechtigkeit,0.001,anteil,0.002,wie,0.002
96,kulturgut,0.002,iban,0.002,standig,0.002,islam,0.002,beteiligter,0.002,gesicherte,0.002,app,0.001,fachen,0.002,zugute,0.002
97,rentensystem,0.002,folgend,0.002,lebensabend,0.002,entweren,0.002,festgelegt,0.002,zensur,0.002,endlich,0.001,erwarmung,0.002,derart,0.002
98,festschreiben,0.002,abgabenlast,0.002,rentenalter,0.002,aufheizung,0.002,wirken,0.002,werbung,0.002,grundsicherung,0.001,emissione,0.002,zugang,0.002


In [116]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(lda_model,corpus,id2word,mds='mmds',R=30)
vis

/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.100266  0.351095       1        1  16.098735
8      0.294055  0.092383       2        1  14.714888
4     -0.241257  0.105460       3        1  13.289037
5      0.057784 -0.272712       4        1  10.686051
7      0.099218  0.185349       5        1  10.054408
0     -0.149893 -0.223674       6        1   9.554732
1     -0.210116 -0.066538       7        1   8.970947
2      0.034076 -0.034137       8        1   8.434545
3      0.216400 -0.137227       9        1   8.196657, topic_info=                 Term        Freq       Total Category  logprob  loglift
4708          spenden  249.000000  249.000000  Default  30.0000  30.0000
4706  alternativefuer  249.000000  249.000000  Default  29.0000  29.0000
4707           paypal  249.000000  249.000000  Default  28.0000  28.0000
173           fordern  132.000000  132.000000  Default  27.0000  27.0000
485              auch  124.000000  124.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
381         erfullung    5.485432    6.257392   Topic9  -5.2993   2.3698
4329   verbotspolitik    5.355748    6.135932   Topic9  -5.3232   2.3655
318         forschung    5.091397    5.860213   Topic9  -5.3739   2.3608
2346           person    5.025875    5.794436   Topic9  -5.3868   2.3591
472         kulturell    4.925560    5.694081   Topic9  -5.4070   2.3565

[309 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
461       1  0.956499          aber
57        8  0.970986   abschaffung
155       7  0.858421   abschiebung
1281      3  0.971486           afd
390       5  0.912756  agrarpolitik
...     ...       ...           ...
515       4  0.933129        zahlen
1101      8  0.971504       zentral
884       5  0.976202          ziel
1151      9  0.969994         zudem
1490      5  0.956571     zunehmend

[284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 5, 6, 8, 1, 2, 3, 4])

In [122]:
bigram_phrases_2=gensim.models.Phrases(data_words_2,min_count=3,threshold=100)
trigram_phases_2=gensim.models.Phrases(bigram_phrases_2[data_words_2],threshold=50)

bigram_2=gensim.models.phrases.Phraser(bigram_phrases_2)
trigram_2=gensim.models.phrases.Phraser(trigram_phases_2)

def make_bigrams_2(texts):
    return [bigram_2[doc] for doc in texts]

def make_trgram_2(texts):
    return [trigram_2[bigram_2[doc]] for doc in texts]

data_bigrams_2=make_bigrams_2(data_words_2)
data_bigrams_trigrams_2=make_trgram_2(data_bigrams_2)

In [123]:
from gensim.models import TfidfModel

id2word_2=corpora.Dictionary(data_bigrams_trigrams_2)

texts_2 = data_bigrams_trigrams_2

corpus_2=[id2word_2.doc2bow(text) for text in texts]

print(corpus_2[3])

tfidf=TfidfModel(corpus_2,id2word=id2word_2)

low_value=0.03
words=[]
words_missing_in_tfdf=[]

for i in tqdm(range(0,len(corpus_2))):
    bow = corpus_2[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops=low_value_words+words_missing_in_tfdf
    for item in drops:
        words.append(id2word_2[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    corpus_2[i]=new_bow


[(5, 1), (6, 1)]


100%|██████████| 1860/1860 [00:00<00:00, 54362.16it/s]


{0: 0.5030000000000003, 1: 0.47600000000000026, 2: 0.49400000000000027, 3: 0.4700000000000003, 4: 0.5220000000000004, 5: 0.4940000000000003, 6: 0.6790000000000003, 7: 0.49500000000000033, 8: 0.5420000000000004}


In [125]:
lda_model_2=gensim.models.ldamodel.LdaModel(
corpus=corpus_2,
id2word=id2word_2,
num_topics=num_topics,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto'
)

In [126]:
dic={}
dic_sum={}
for idx, topic in lda_model_2.print_topics(num_topics,num_words=100):
    elements=[]
    percent=[]

    for e in topic.split('+'):
        elements.append(e.split('*')[1].replace('"','').strip())
        percent.append(float(e.split('*')[0].replace('"','').strip()))

    dic[str(idx)+'_word']=elements
    dic[str(idx)+'_per']=percent
    dic_sum[idx]=sum(percent)
print(dic_sum)

{0: 0.5910000000000004, 1: 0.5830000000000004, 2: 0.5540000000000004, 3: 0.6260000000000003, 4: 0.5610000000000004, 5: 0.6270000000000003, 6: 0.5360000000000004, 7: 0.6060000000000004, 8: 0.5920000000000004}


In [127]:
pd.DataFrame(data=dic)

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,5_word,5_per,6_word,6_per,7_word,7_per,8_word,8_per
0,burger,0.067,verbraucher,0.023,forderung,0.035,land,0.040,ausbau,0.046,landwirtschaft,0.041,eu,0.028,familie,0.042,identitat,0.024
1,sicherheit,0.026,tier,0.022,klima_energie_technik_digitalisierung,0.031,mensch,0.032,kommune,0.025,kind,0.038,heimat,0.024,hand,0.022,natur,0.018
2,technik,0.020,produkt,0.020,aufgabe,0.025,versorgung,0.029,schule,0.021,wald,0.031,steuer,0.015,landwirt,0.020,standard,0.018
3,grundgesetz,0.016,wirkung,0.015,abschaffung,0.020,digitalisierung,0.024,nutzung,0.020,kultur,0.026,stream,0.015,bildung,0.019,staat,0.017
4,mobilitat_infrastruktur,0.014,neubau,0.014,integration,0.015,energie,0.022,gebiet,0.019,sprache,0.022,beitrag,0.013,bevolkerung,0.017,schutz,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,verringerung,0.002,dienstleistung,0.002,eeg,0.002,kontext,0.002,zeitpunkt,0.002,server,0.002,objektwert,0.002,vorstellung,0.002,hafen,0.002
96,fachen,0.002,rechtsrahmen,0.002,gez,0.002,gewahrleistung,0.002,ergru,0.002,weg,0.002,kapitalersatz,0.002,ordnungsrahmen,0.002,schlepptau,0.002
97,offentlich,0.002,zertifizierung,0.002,umlage,0.002,datenbank,0.002,erde,0.002,versuch,0.002,umfeld,0.002,medizinstudent,0.002,risikobewertung,0.002
98,fortbewegung,0.002,milliarde,0.002,haushaltsabgabe,0.002,patientendate,0.002,atmosphare,0.002,niveau,0.002,laufzeit,0.002,ubernahme,0.002,jugendamt,0.002


In [128]:
pyLDAvis.enable_notebook()
vis=gensimvis.prepare(lda_model_2,corpus_2,id2word_2,mds='mmds',R=20)
vis

/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/NLP_spacy

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.084393  0.253244       1        1  15.218594
5      0.295024  0.097807       2        1  13.843335
3     -0.281182  0.146933       3        1  12.570568
7      0.061635 -0.287902       4        1  11.156080
0      0.118528  0.267278       5        1  10.574552
6     -0.148775 -0.229363       6        1   9.361678
1     -0.248241 -0.071818       7        1   9.125868
2      0.044516 -0.033135       8        1   9.105241
4      0.242888 -0.143043       9        1   9.044084, topic_info=                Term       Freq      Total Category  logprob  loglift
95            burger  50.000000  50.000000  Default  20.0000  20.0000
260   landwirtschaft  39.000000  39.000000  Default  19.0000  19.0000
116          familie  33.000000  33.000000  Default  18.0000  18.0000
122             kind  37.000000  37.000000  Default  17.0000  17.0000
45              land  35.000000  35.000000  Default  16.0000  16.0000
...              ...        ...        ...      ...      ...      ...
2532        fahrzeug   5.490294   6.122139   Topic9  -4.7394   2.2941
1096      bundeswehr   5.458762   6.089073   Topic9  -4.7452   2.2938
1031      einhaltung   5.260473   5.889508   Topic9  -4.7822   2.2901
1068       bedeutung   5.252369   5.881148   Topic9  -4.7838   2.2900
2433            netz   4.806223   5.444917   Topic9  -4.8725   2.2783

[204 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
43        8  0.914418   abschaffung
104       7  0.937063   abschiebung
1685      2  0.939075    abtreibung
701       2  0.949792           afd
261       6  0.938143  agrarpolitik
...     ...       ...           ...
2353      8  0.846014  wohnbebauung
227       5  0.931444      wohngeld
1445      7  0.940159          zahl
499       2  0.324356          ziel
499       4  0.648712          ziel

[188 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 4, 8, 1, 7, 2, 3, 5])

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg")
text_list=[]
lemma_list=[]
pos_list=[]
tag_list=[]
dep_list=[]
shape_list=[]
alpha_list=[]
stop_list=[]

for text in tqdm(df.text.tolist()):
    doc = nlp(text)
    text_list_word=[]
    lemma_list_word=[]
    pos_list_word=[]
    tag_list_word=[]
    dep_list_word=[]
    shape_list_word=[]
    alpha_list_word=[]
    stop_list_word=[]
    for token in doc:
        text_list_word.append(token.text)
        lemma_list_word.append(token.lemma_)
        pos_list_word.append(token.pos_)
        tag_list_word.append(token.tag_)
        dep_list_word.append(token.dep_)
        shape_list_word.append(token.shape_)
        alpha_list_word.append(token.is_alpha)
        stop_list_word.append(token.is_stop)
    text_list.append(text_list_word)
    lemma_list.append(lemma_list_word)
    pos_list.append(pos_list_word)
    tag_list.append(tag_list_word)
    dep_list.append(dep_list_word)
    shape_list.append(shape_list_word)
    alpha_list.append(alpha_list_word)
    stop_list.append(stop_list_word)
    #print(text_list)
d = {'text':text,'text_list': text_list, 'lemma_list': lemma_list,'pos_list': pos_list, 'tag_list': tag_list,'dep_list': dep_list, 'shape_list': shape_list,'alpha_list': alpha_list, 'stop_list': stop_list,'gender':df.gender.tolist(),'sign':df.sign.tolist(),'topic':df.topic.tolist()}
df_text_spacy=pd.DataFrame(data=d)
    
